In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

# Definindo os vetores de amostras e resultados
x1 = np.array([-0.6508, -1.4492, 2.085, 0.2626, 0.6418, 0.2569, 1.1555, 0.0914, 0.0121, -0.0429, 0.434, 0.2735, 0.4839, 0.4089, 1.4391, -0.9115, 0.3654, 0.2144, 0.2013, 0.6483, -0.1147, -0.797, -1.0625, 0.5307, -1.22, 0.3957, -0.1013, 2.4482, 2.0149, 0.2012])
x2 = np.array([0.1097, 0.8896, 0.6876, 1.1476, 1.0234, 0.673, 0.6043, 0.3399, 0.5256, 0.466, 0.687, 1.0287, 0.4851, -0.1267, 0.1614, -0.1973, 1.0475, 0.7515, 1.0014, 0.2183, 0.2242, 0.8795, 0.6366, 0.1285, 0.7777, 0.1076, 0.5989, 0.9455, 0.6192, 0.2611])
x3 = np.array([4.0009, 4.4005, 12.071, 7.7985, 7.0427, 8.3265, 7.4446, 7.0677, 4.6316, 5.4323, 8.2287, 7.1934, 7.485, 5.5019, 8.5843, 2.1962, 7.4858, 7.1699, 6.5489, 5.8991, 7.2435, 3.8762, 2.4707, 5.6883, 1.7252, 5.6623, 7.1812, 11.2095, 10.9263, 5.4631])
results = np.array([-1.0, -1.0, -1.0, 1.0, 1.0, -1.0, 1.0, -1.0, 1.0, 1.0, -1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 1.0, 1.0, -1.0, -1.0, 1.0, -1.0, 1.0])

# Empilhando os vetores x1, x2 e x3 para formar uma matriz de entradas de 30x3
X = np.column_stack((np.ones(len(x1)), x1, x2, x3))  # Adicionando termo de viés

# FUNÇÃO ATIVAÇÃO -> DEGRAU BIOLAR
def sinal(x):
    return np.where(x >= 0, 1, -1)

#Treinanemento do modelo
def treinar_perceptron(X, y, taxa_aprendizado=0.01, max_epocas=1000):
    n_amostras, n_features = X.shape
    W = np.random.uniform(-1, 1, n_features) #GERA VALORES ALEATÓRIOS PARA OS PESOS DE ENTRADA ENTRE -1 E 1
    W_inicial = W.copy()  # Salvando os pesos iniciais
    
    for epoca in range(max_epocas):
        erro = False
        for i in range(n_amostras):
            u = np.dot(W, X[i])
            y_pred = sinal(u)
            
            if y_pred != y[i]:
                W += taxa_aprendizado * (y[i] - y_pred) * X[i] #função para atualizar os novos pesos
                erro = True
        
        if not erro:
            print(f"Convergência alcançada na época {epoca + 1}")
            break
    
    return W_inicial, W, epoca + 1

# Função para testar o perceptron treinado
def testar_perceptron(X, W):
    return sinal(np.dot(X, W))

# Função para salvar os resultados em Excel
def salvar_resultados(W_inicial, W_final, previsoes_teste, epocas, taxa_aprendizado=0.01, nome_arquivo=None):
    # Criar nome do arquivo com timestamp se não fornecido
    if nome_arquivo is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        nome_arquivo = f"resultados_perceptron_5.xlsx"
    
    # Criar DataFrame com as informações do treinamento
    df_info_treinamento = pd.DataFrame({
        'Parâmetro': ['Número de Épocas', 'Taxa de Aprendizado'],
        'Valor': [epocas, taxa_aprendizado]
    })
    
    # Criar DataFrame com os pesos
    df_pesos = pd.DataFrame({
        'Componente': ['Bias', 'X1', 'X2', 'X3'],
        'Peso_Inicial': W_inicial,
        'Peso_Final': W_final,
        'Diferença': W_final - W_inicial
    })
    
    # Criar DataFrame com as classificações
    df_classificacoes = pd.DataFrame({
        'Amostra': range(1, len(previsoes_teste) + 1),
        'Classificacao_Numerica': previsoes_teste,
        'Classificacao': ['CLASSE A' if p == -1 else 'CLASSE B' for p in previsoes_teste]
    })
    
    # Salvar em arquivo Excel
    with pd.ExcelWriter(nome_arquivo) as writer:
        df_info_treinamento.to_excel(writer, sheet_name='Informações', index=False)
        df_pesos.to_excel(writer, sheet_name='Pesos', index=False)
        df_classificacoes.to_excel(writer, sheet_name='Classificações', index=False)
    
    print(f"\nResultados salvos em: {nome_arquivo}")
    
    return nome_arquivo

def testar_com_novos_dados():
    # Dados de teste
    x1_teste = np.array([-0.3565, -0.7842, 0.3012, 0.7757, 0.1570, -0.7014, 0.3748, -0.6920, -1.3970, -1.8842])
    x2_teste = np.array([0.0620, 1.1267, 0.5611, 1.0648, 0.8028, 1.0316, 0.1536, 0.9404, 0.7141, -0.2805])
    x3_teste = np.array([5.9891, 5.5912, 5.8234, 8.0677, 6.3040, 3.6005, 6.1537, 4.4058, 4.9263, 1.2548])
    
    # Criar matriz de teste com o termo de viés (ones)
    X_teste = np.column_stack((np.ones(len(x1_teste)), x1_teste, x2_teste, x3_teste))
    
    # Fazendo previsões
    previsoes_teste = testar_perceptron(X_teste, W_final)
    
    print("\nResultados do teste com novos dados:")
    print("Classificação das amostras:")
    for i, previsao in enumerate(previsoes_teste):
        classe = "CLASSE A" if previsao == -1 else "CLASSE B"
        print(f"Amostra {i+1}: {classe}")
    
    return previsoes_teste

# Definir taxa de aprendizado
taxa_aprendizado = 0.01

# Treinamento do perceptron
W_inicial, W_final, epocas = treinar_perceptron(X, results, taxa_aprendizado=taxa_aprendizado)

print(f"Número de épocas: {epocas}")
print(f"\nPesos iniciais: {W_inicial}")
print(f"Pesos finais: {W_final}")

# Teste com os dados de treinamento
previsoes = testar_perceptron(X, W_final)
acuracia = np.mean(previsoes == results) * 100
print(f"\nAcurácia do modelo nos dados de treinamento: {acuracia:.2f}%")

# Testando com novos dados e salvando resultados
previsoes_teste = testar_com_novos_dados()
nome_arquivo = salvar_resultados(W_inicial, W_final, previsoes_teste, epocas, taxa_aprendizado)

Convergência alcançada na época 432
Número de épocas: 432

Pesos iniciais: [ 0.5286698   0.74428011 -0.231342    0.82554738]
Pesos finais: [ 3.1686698   1.60451011  2.548388   -0.75312662]

Acurácia do modelo nos dados de treinamento: 100.00%

Resultados do teste com novos dados:
Classificação das amostras:
Amostra 1: CLASSE A
Amostra 2: CLASSE B
Amostra 3: CLASSE B
Amostra 4: CLASSE B
Amostra 5: CLASSE B
Amostra 6: CLASSE B
Amostra 7: CLASSE A
Amostra 8: CLASSE B
Amostra 9: CLASSE A
Amostra 10: CLASSE A

Resultados salvos em: resultados_perceptron_5.xlsx
